# IMD3004 - IA Generativa

### Professor: Dr. Leonardo Enzo Brito da Silva

### Aluno: João Antonio Costa Paiva Chagas

## Tarefa: Treinar uma GAN com o conjunto *Two Moons* do scikit-learn

Utilize o código acima como base e adapte-o para o seguinte problema:

1. Gere o conjunto de dados *Two Moons* utilizando a função `make_moons` do [`scikit-learn`](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_moons.html):

```python
from sklearn.datasets import make_moons
X, y = make_moons(n_samples=4096, noise=0.1, random_state=42)
```

Obs.: lembre-se de normalizar os dados.

2. Substitua os dados de treinamento originais do código pelo conjunto *Two Moons*.  
3. Treine uma GAN (com gerador e discriminador definidos no código) para aprender a distribuição do *Two Moons* (se necessário faça mudanças nas arquiteturas das redes neurais). **Não utilize parada antecipada: treine a GAN por 1000 épocas.**
4. Registre a evolução das perdas do gerador e do discriminador ao longo do treinamento.  
5. Gere gráficos comparando os dados reais (*Two Moons*) e as amostras criadas pelo gerador em diferentes épocas.  
6. Salve o modelo treinado e mostre como carregar e gerar novas amostras a partir dele.  

**Entregáveis:**  
1. Notebook .ipynb contendo:  
- O código adaptado e comentado.  
2. Relatório em .pdf
- Gráficos que mostrem o processo de treinamento.  
- Uma breve discussão sobre o treinamento e arquitetura da GAN, bem como sobre os resultados obtidos.

### Importações:

In [ ]:
import os
import torch
import torchvision
import numpy as np
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from sklearn.datasets import make_moons
import torchvision.transforms as transforms
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset

### Ambiente:

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
if not os.path.exists('files'):
    os.makedirs('files')

### Dados:

In [ ]:
def prepare_two_moons_data(batch_size):
    """Gera, escala e carrega a base de dados Two Moons."""
    X, y = make_moons(n_samples=4096, noise=0.1, random_state=42)
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    training_data = torch.tensor(X_scaled, dtype=torch.float32)
    train_loader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
    return train_loader

In [ ]:
def prepare_mnist_data(batch_size):
    """Baixa, transforma e carrega a base de dados MNIST."""
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])
    train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=2,
        pin_memory=True
    )
    return train_loader

In [ ]:
def preparar_dados(dataset_choice, batch_size):
    """Chama a função apropriada para preparar a base de dados escolhida."""
    if dataset_choice == 'two_moons':
        return prepare_two_moons_data(batch_size)
    elif dataset_choice == 'mnist':
        return prepare_mnist_data(batch_size)
    else:
        raise ValueError("Escolha inválida. Escolha 'two_moons' ou 'mnist'.")

### Plotando:

In [ ]:
def generate_samples(gen_model, device, params):
    """Gera um lote de amostras pelo gerador."""
    with torch.no_grad():
        gen_model.eval()
        noise = torch.randn(256, params['z_size'], device=device)
        generated_samples = gen_model(noise).cpu().numpy()
        gen_model.train()
    return generated_samples

In [ ]:
def plot_two_moons_samples(epoch, generated_samples, train_loader, dataset_choice):
    """Cria e salva um scatter plot para a base de dados duas luas."""
    plt.figure(figsize=(8, 8))
    real_data = train_loader.dataset.numpy()
    plt.scatter(real_data[:, 0], real_data[:, 1], c='r', alpha=0.1, label='Amostras Reais')
    plt.scatter(generated_samples[:, 0], generated_samples[:, 1], c='g', alpha=0.6, label='Amostras Geradas')
    plt.title(f'Época {epoch}')
    plt.legend()
    plt.savefig(f"files/{dataset_choice}_p{epoch}.png")
    plt.close()

In [ ]:
def plot_mnist_samples(epoch, generated_samples, dataset_choice):
    """Cria e salva um grid de imagem para a base de dados MNIST."""
    fig, axes = plt.subplots(4, 4, figsize=(8, 8))
    fig.suptitle(f'Época {epoch}', fontsize=16)
    for i, ax in enumerate(axes.flatten()):
        if i < 16:
            img = generated_samples[i].reshape(28, 28)
            ax.imshow(img, cmap='gray')
        ax.axis('off')
    plt.savefig(f"files/{dataset_choice}_p{epoch}.png")
    plt.close()

In [ ]:
def log_and_save_samples(epoch, gen_model, device, params, dataset_choice, train_loader):
    """Guarda os plots das amostras geradas."""
    print(f"Época {epoch}/{params['num_epochs']} | Logging samples...")

    # 1. Gera amostras
    generated_samples = generate_samples(gen_model, device, params)

    # 2. Plota e salva baseado na base de dados
    if dataset_choice == 'two_moons':
        plot_two_moons_samples(epoch, generated_samples, train_loader, dataset_choice)
    elif dataset_choice == 'mnist':
        plot_mnist_samples(epoch, generated_samples, dataset_choice)

In [ ]:
def plot_two_moons_evolution(dataset_choice='two_moons', epochs_to_show=[25, 100, 500, 1000]):
    """Carrega os plots salvos para o Two Moons."""
    print("Mostrando a evolução das amostras para o dataset Two Moons:")
    fig, axes = plt.subplots(1, len(epochs_to_show), figsize=(20, 5))
    for i, epoch in enumerate(epochs_to_show):
        file_path = f"files/{dataset_choice}_p{epoch}.png"
        if os.path.exists(file_path):
            img = plt.imread(file_path)
            axes[i].imshow(img)
            axes[i].set_title(f"Resultado na Época {epoch}")
        else:
            axes[i].set_title(f"Arquivo não encontrado")
        axes[i].axis('off')
    plt.tight_layout()
    plt.show()

In [ ]:
def plot_mnist_evolution(epoch_samples):
    """Plota a evolução das amostras MNIST coletadas durante o treino."""
    print("Mostrando a evolução das amostras para o dataset MNIST:")
    fig, axes = plt.subplots(len(epoch_samples), 10, figsize=(15, len(epoch_samples) * 1.5))
    if len(epoch_samples) == 1: axes = np.array([axes])
    fig.suptitle("Evolução das Amostras Geradas", fontsize=16, y=1.02)
    for row, data in enumerate(epoch_samples):
        epoch_num, samples = data['epoch'], data['samples']
        for col in range(10):
            ax = axes[row, col]
            img = samples[col].reshape(28, 28)
            ax.imshow(img, cmap='gray')
            ax.axis('off')
            if col == 0:
                ax.text(-0.1, 0.5, f'Época {epoch_num}', ha='right', va='center', transform=ax.transAxes, fontsize=12)
    plt.show()

In [ ]:
def plot_losses(g_losses, d_losses):
    """Plota as perdas."""
    plt.figure(figsize=(10, 5))
    plt.plot(g_losses, label='Generator Loss')
    plt.plot(d_losses, label='Discriminator Loss')
    plt.title('Evolução das Perdas da GAN')
    plt.xlabel('Época')
    plt.ylabel('Perda (BCELoss)')
    plt.legend()
    plt.grid(True)
    plt.show()

In [ ]:
def plot_final_samples(generated_samples, dataset_choice, train_loader):
    """Exibe as amostras geradas finais."""
    print("Amostras geradas com o modelo final carregado:")

    if dataset_choice == 'two_moons':
        plt.figure(figsize=(8, 8))
        real_data = train_loader.dataset.numpy()
        plt.scatter(real_data[:, 0], real_data[:, 1], c='r', alpha=0.1)
        plt.scatter(generated_samples[:, 0], generated_samples[:, 1], c='g', alpha=0.6)
        plt.title("Amostras Finais (Two Moons)")
        plt.show()

    elif dataset_choice == 'mnist':
        fig, axes = plt.subplots(4, 4, figsize=(8, 8))
        fig.suptitle("Amostras Finais (MNIST)", fontsize=16)
        for i, ax in enumerate(axes.flatten()):
            if i < 16:
                img = generated_samples[i].reshape(28, 28)
                ax.imshow(img, cmap='gray')
            ax.axis('off')
        plt.show()

### GAN:

#### Hiperparâmetros:

In [ ]:
def get_hyperparameters(dataset_choice):
    """Retorna um dicionário de hiperparâmetros baseado na base de dados escolhida."""
    if dataset_choice == 'two_moons':
        return {
            'z_size': 2, 'image_size': 2, 'lr': 0.0005, 'num_epochs': 1000, 'batch_size': 128
        }
    elif dataset_choice == 'mnist':
        return {
            'z_size': 100, 'image_size': 28 * 28, 'lr': 0.0002, 'num_epochs': 50, 'batch_size': 64
        }

#### Gerador:

In [ ]:
def get_generator(dataset_choice, z_size, image_size):
    """Retorna a arquitetura correta do gerador baseada na escolha da base de dados."""
    if dataset_choice == 'two_moons':
        return nn.Sequential(
            nn.Linear(z_size, 16),
            nn.ReLU(),
            nn.Linear(16, 32),
            nn.ReLU(),
            nn.Linear(32, image_size)
        )
    elif dataset_choice == 'mnist':
        return nn.Sequential(
            nn.Linear(z_size, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 512),
            nn.LeakyReLU(0.2),
            nn.Linear(512, image_size),
            nn.Tanh()
          )

In [ ]:
def train_generator_step(disc_model, gen_model, optim_g, loss_fn, batch_size, device, z_size):
    optim_g.zero_grad()                                                         # Zera os gradientes acumulados do gerador.

    # Gera amostras falsas
    noise = torch.randn(batch_size, z_size, device=device)                      # Gera ruído aleatório
    fake_samples = gen_model(noise)                                             # Gerador produz imagens falsas a partir do ruído.

    # Calcula perda baseado na saída do discriminador
    g_output = disc_model(fake_samples)                                         # Avalia as amostras geradas no discriminador.
    g_loss = loss_fn(g_output, torch.ones_like(g_output))                       # Calcula a perda do gerador

    # Atualiza Pesos
    g_loss.backward()                                                           # Propaga os gradientes no grafo do gerador.
    optim_g.step()                                                              # Atualiza os pesos do gerador.

    return g_loss.item()

#### Discriminador:

In [ ]:
def get_discriminator(dataset_choice, image_size):
    """Retorna a arquitetura correta do discriminador baseada na escolha da base de dados."""
    if dataset_choice == 'two_moons':
        return nn.Sequential(
            nn.Linear(image_size, 256),
            nn.ReLU(), nn.Dropout(0.3),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 1),
            nn.Sigmoid()
        )
    elif dataset_choice == 'mnist':
        return nn.Sequential(
            nn.Linear(image_size, 512),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

In [ ]:
def train_discriminator_step(disc_model, gen_model, optim_d, loss_fn, real_samples, device, z_size, dataset_choice):
    optim_d.zero_grad()                                                         # Zera os gradientes acumulados do discriminador
    current_batch_size = real_samples.size(0)                                   # Obtém o tamanho do lote (número de amostras).

    if dataset_choice == 'mnist':                                               # Achata imagens se for MNIST
        real_samples = real_samples.view(current_batch_size, -1)
    real_samples = real_samples.to(device)

    # Calcula a perda nas amostras reais
    d_real_output = disc_model(real_samples)
    d_real_loss = loss_fn(d_real_output, torch.ones_like(d_real_output))        # Calcula a perda do discriminador para o lote real.

    # Calcula a perda nas amostras falsas
    noise = torch.randn(current_batch_size, z_size, device=device)              # Gera ruído aleatório
    fake_samples = gen_model(noise)                                             # Gerador produz imagens falsas a partir do ruído.
    d_fake_output = disc_model(fake_samples.detach())                           # Passa as imagens falsas pelo discriminador e obtém a saída
    d_fake_loss = loss_fn(d_fake_output, torch.zeros_like(d_fake_output))       # Calcula a perda do discriminador para o lote falso.

    # Soma as perdas e atualiza os pesos
    d_loss = (d_real_loss + d_fake_loss) / 2                                    # Combina as perdas do lote real e falso.
    d_loss.backward()                                                           # Propaga os gradientes da perda total.
    optim_d.step()                                                              # Atualiza os parâmetros do discriminador.

    return d_loss.item()

#### Treinamento:

In [ ]:
def setup_training_components(gen_model, disc_model, device, params, dataset_choice):
    """Inicializa todos os componentes necessários para o treinamento."""
    components = {
        'loss_fn': nn.BCELoss(),
        'optim_d': optim.Adam(disc_model.parameters(), lr=params['lr']),
        'optim_g': optim.Adam(gen_model.parameters(), lr=params['lr']),
        'g_losses': [],
        'd_losses': [],
        'epoch_samples_mnist': [],
        'fixed_noise_mnist': None
    }
    if dataset_choice == 'mnist':
        components['fixed_noise_mnist'] = torch.randn(64, params['z_size'], device=device)
    return components

In [ ]:
def train_one_epoch(gen_model, disc_model, train_loader, loss_fn, optim_d, optim_g, device, params, dataset_choice):
    """Executa uma única época de treinamento para a GAN."""
    epoch_g_loss, epoch_d_loss = 0.0, 0.0
    for batch in train_loader:
        real_data = batch[0] if dataset_choice == 'mnist' else batch

        d_loss = train_discriminator_step(disc_model, gen_model, optim_d, loss_fn, real_data, device, params['z_size'], dataset_choice)
        g_loss = train_generator_step(disc_model, gen_model, optim_g, loss_fn, real_data.size(0), device, params['z_size'])

        epoch_d_loss += d_loss
        epoch_g_loss += g_loss

    avg_g_loss = epoch_g_loss / len(train_loader)
    avg_d_loss = epoch_d_loss / len(train_loader)

    return avg_g_loss, avg_d_loss

In [ ]:
def handle_epoch_logging(epoch, gen_model, device, params, dataset_choice, train_loader, components):
    """Lida com o logging e salvamento de amostras no final de uma época."""
    g_losses = components['g_losses']
    d_losses = components['d_losses']
    epoch_samples_mnist = components['epoch_samples_mnist']
    fixed_noise_mnist = components['fixed_noise_mnist']

    if dataset_choice == 'two_moons':
        if epoch % 25 == 0 or epoch == 1:
            log_and_save_samples(epoch, gen_model, device, params, dataset_choice, train_loader)

    elif dataset_choice == 'mnist':
        if epoch % 5 == 0 or epoch == 1 or epoch == params['num_epochs']:
            print(f"Época {epoch}/{params['num_epochs']} | G Loss: {g_losses[-1]:.4f} | D Loss: {d_losses[-1]:.4f}")
            gen_model.eval()
            with torch.no_grad():
                samples = gen_model(fixed_noise_mnist).cpu()
                epoch_samples_mnist.append({'epoch': epoch, 'samples': samples})
            gen_model.train()

In [ ]:
def treinar_gan(gen_model, disc_model, train_loader, device, params, dataset_choice):
    """Orquestra o processo de treinamento da GAN."""

    # 1. Inicialização
    components = setup_training_components(gen_model, disc_model, device, params, dataset_choice)

    print(f"Começo do treino para o dataset: {dataset_choice}")
    for epoch in range(1, params['num_epochs'] + 1):

        # 2. Treinamento de uma época
        avg_g_loss, avg_d_loss = train_one_epoch(
            gen_model, disc_model, train_loader, components['loss_fn'],
            components['optim_d'], components['optim_g'], device, params, dataset_choice
        )

        components['g_losses'].append(avg_g_loss)
        components['d_losses'].append(avg_d_loss)

        # 3. Logging da época
        handle_epoch_logging(epoch, gen_model, device, params, dataset_choice, train_loader, components)

    print("Fim do treino.")
    return components['g_losses'], components['d_losses'], components['epoch_samples_mnist']


### Salvando, carregando e testando:

In [ ]:
def save_model(model, dataset_choice):
    """Salva o modelo do gerador treinado num arquivo."""
    model_path = f'files/{dataset_choice}_generator.pt'
    torch.jit.script(model).save(model_path)
    print(f"\nModelo final salvo em: {model_path}")
    return model_path

In [ ]:
def load_model(model_path, device):
    """Carrega um modelo salvo e o configura para o modo de avaliação."""
    loaded_model = torch.jit.load(model_path, map_location=device)
    loaded_model.eval()
    return loaded_model

In [ ]:
def save_load_test_model(model, device, params, dataset_choice, train_loader):
    """Salva o modelo final, carrega de volta, e gera um plot de amostras finais."""
    # 1. Salva o modelo
    model_path = save_model(model, dataset_choice)

    # 2. Carrega o modelo
    loaded_model = load_model(model_path, device)

    # 3. Gera e plota as amostras finais
    final_samples = generate_samples(loaded_model, device, params)
    plot_final_samples(final_samples, dataset_choice, train_loader)

### Executando:

In [ ]:
# ==========================================================
# --- CONTROLE PRINCIPAL: Escolha 'two_moons' ou 'mnist' ---
dataset_choice = 'two_moons'
# ==========================================================

# 1. Pega os hiperparâmetros corretos
params = get_hyperparameters(dataset_choice)

# 2. Prepara os dados
train_loader = preparar_dados(dataset_choice, params['batch_size'])

# 3. Cria os modelos corretos
gerador = get_generator(dataset_choice, params['z_size'], params['image_size']).to(device)
discriminador = get_discriminator(dataset_choice, params['image_size']).to(device)

# 4. Treina a GAN
g_losses, d_losses, epoch_samples = treinar_gan(gerador, discriminador, train_loader, device, params, dataset_choice)

# 5.Exibe os resultados
plot_losses(g_losses, d_losses)

if dataset_choice == 'two_moons':
    plot_two_moons_evolution()
elif dataset_choice == 'mnist':
    plot_mnist_evolution(epoch_samples)

save_load_test_model(gerador, device, params, dataset_choice, train_loader)

In [ ]:
from torchsummary import summary

dataset_choice = 'mnist'

params = get_hyperparameters(dataset_choice)
train_loader = preparar_dados(dataset_choice, params['batch_size'])

gerador = get_generator(dataset_choice, params['z_size'], params['image_size']).to(device)
discriminador = get_discriminator(dataset_choice, params['image_size']).to(device)

summary(gerador, input_size=(1, params['z_size']))

summary(discriminador, input_size=(1, np.prod(params['image_size'])))

### Para o lab 6:

In [ ]:
# ==========================================================
# --- CONTROLE PRINCIPAL: Escolha 'two_moons' ou 'mnist' ---
dataset_choice = 'mnist'
# ==========================================================

# 1. Pega os hiperparâmetros corretos
params = get_hyperparameters(dataset_choice)

# 2. Prepara os dados
train_loader = preparar_dados(dataset_choice, params['batch_size'])

# 3. Cria os modelos corretos
gerador = get_generator(dataset_choice, params['z_size'], params['image_size']).to(device)
discriminador = get_discriminator(dataset_choice, params['image_size']).to(device)

# 4. Treina a GAN
g_losses, d_losses, epoch_samples = treinar_gan(gerador, discriminador, train_loader, device, params, dataset_choice)

# 5.Exibe os resultados
plot_losses(g_losses, d_losses)

if dataset_choice == 'two_moons':
    plot_two_moons_evolution()
elif dataset_choice == 'mnist':
    plot_mnist_evolution(epoch_samples)

save_load_test_model(gerador, device, params, dataset_choice, train_loader)